In [2]:
import FinanceDataReader as fdr
import time
import pandas as pd
import numpy as np
import time, json, requests 
from datetime import datetime, timedelta, date
import pymysql
import sqlalchemy
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import time
#비슷한 코드들에는 주석을 적지 않았습니다.

In [3]:
#krx에서 데이터를 가져옵니다.
stock_list = fdr.StockListing('KRX').dropna()

In [4]:
stock_list

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,김세완,http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,"박대현, 손삼달",http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도
...,...,...,...,...,...,...,...,...,...,...
7849,189980,KOSDAQ,흥국에프엔비,비알코올음료 및 얼음 제조업,"과일음료(에이드베이스, 스무디, 착즙쥬스 등)",2015-08-07,12월,"박철범, 오길영",http://www.hyungkuk.com,서울특별시
7850,000540,KOSPI,흥국화재,보험업,손해보험,1974-12-05,12월,임규준,http://www.insurance.co.kr,서울특별시
7853,003280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),1976-06-29,12월,이환구,http://www.heung-a.com,서울특별시
7854,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,2000-02-03,12월,"정영균, 이목운, 허철호 (각자대표)",http://www.heerim.com,서울특별시


In [5]:
#krx에서 얻어온 stock_list에서 code만을 추출해 symbol_code에 삽입합니다.
symbol_code = []
for i in range(len(stock_list)):
    symbol_code.append(stock_list['Symbol'].values[i])
symbol_code

['060310',
 '095570',
 '006840',
 '054620',
 '265520',
 '211270',
 '027410',
 '282330',
 '032790',
 '138930',
 '001460',
 '013720',
 '001040',
 '079160',
 '035760',
 '311690',
 '000120',
 '011150',
 '097950',
 '051500',
 '058820',
 '023460',
 '056730',
 '065770',
 '083660',
 '000590',
 '012030',
 '016610',
 '005830',
 '000990',
 '139130',
 '001530',
 '000210',
 '001880',
 '068790',
 '004840',
 '241520',
 '155660',
 '069730',
 '017940',
 '245620',
 '037370',
 '278990',
 '365550',
 '050120',
 '084180',
 '383220',
 '007700',
 '214270',
 '130500',
 '114090',
 '083450',
 '006360',
 '001250',
 '007070',
 '297890',
 '078150',
 '012630',
 '039570',
 '089470',
 '294870',
 '267250',
 '097230',
 '195940',
 '028300',
 '003580',
 '343090',
 '067630',
 '047920',
 '011200',
 '036640',
 '082740',
 '335870',
 '388790',
 '003560',
 '095340',
 '099520',
 '175330',
 '950170',
 '234080',
 '067290',
 '001060',
 '096760',
 '035900',
 '318000',
 '024840',
 '105560',
 '024120',
 '002380',
 '021320',
 '344820',

In [13]:
#getPER 함수 (연간 데이터)
def getPER(date):
    per_arr = []   #받아온 날짜로 per을 구해서 저장합니다.
    name_arr = []  #종목이름도 같이 저장합니다.
    date_arr = []  #받아온 날짜도 저장합니다.
    code = []      #코드도 마찬가지로 저장합니다.
    for i in range(len(symbol_code)):   #symbol_code를 돌면서 한 종목씩 per을 구해줍니다.
        conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
        curs = conn.cursor()
        stock_code = symbol_code[i]
        sql = f"SELECT stock_code, period, netinc1 FROM income_statement WHERE stock_code = '{stock_code}' AND period = '{date}'"
        curs.execute(sql)
        income_statement = curs.fetchall()   #손익계산서에서 받아온 코드, 날짜, 당기순이익을 변수에 저장합니다.

        curs = conn.cursor()
        sql = f"SELECT stock_code, number FROM stock_number WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        stock_number = curs.fetchall()   #주식수 테이블에서 받아온 코드, 주식수를 변수에 저장합니다.

        curs = conn.cursor()
        sql = f"SELECT Code, Date, Close, Name FROM asp WHERE Code = '{stock_code}' AND Date = '{date}'"
        curs.execute(sql)
        asp = curs.fetchall()   #수정주가 테이블에서 받아온 코드, 날짜, 주가, 이름을 변수에 저장합니다.

        if len(income_statement) == 0:   #만일 코드와 인자로받은 날짜에 해당하는 데이터가 없는경우는 넘어갑니다.
            continue
        if len(asp) == 0:
            continue
        netinc1 = round(float(income_statement[0][2]) * 100000000, 2)
        eps = round(netinc1 / stock_number[0][1], 2)   #(당기순이익 / 주식수)계산을 통해 eps를 구해줍니다.
        if eps <= 0:   #eps가 0이하일 경우는 넘어갑니다.
            continue
        #각 종목당 per, 이름, 날짜, 코드를 하나씩 저장해줍니다.
        per_arr.append(round((asp[0][2] / eps), 2))
        name_arr.append(asp[0][3])
        date_arr.append(asp[0][1])
        code.append(stock_code)

    conn.close()
    #배열로 저장했던 값들을 데이터프레임으로 변환합니다.
    per = pd.DataFrame(data = name_arr, index = code, columns=['name'])
    per['per'] = per_arr
    per['date'] = date_arr
    per = per.dropna()
    per = per[['per', 'date', 'name']]
    #per에 대해 오름차순으로 정렬한후 반환합니다.
    per = per.sort_values('per',ascending=True)
    
    return per

In [12]:
#getPBR 함수 (연간 데이터)
def getPBR(date):
    pbr_arr = []
    name_arr = []
    date_arr = []
    code = []
    for i in range(len(symbol_code)):
        conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
        curs = conn.cursor()
        stock_code = symbol_code[i]
        sql = f"SELECT stock_code, period, Controlling_Equity_Total FROM balance_statement WHERE stock_code = '{stock_code}' AND period = '{date}'"
        curs.execute(sql)
        balance_statement = curs.fetchall()   #재무상태표에서 받아온 코드, 날짜, 자본을 변수에 저장합니다.

        curs = conn.cursor()
        sql = f"SELECT stock_code, number FROM stock_number WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        stock_number = curs.fetchall()

        curs = conn.cursor()
        sql = f"SELECT Code, Date, Close, Name FROM asp WHERE Code = '{stock_code}' AND Date = '{date}'"
        curs.execute(sql)
        asp = curs.fetchall()

        if len(asp) == 0:
            continue
        if len(balance_statement) == 0:
            continue
        Controlling_Equity_Total = round(float(balance_statement[0][2]) * 100000000, 2)
        bps = round(Controlling_Equity_Total / stock_number[0][1], 2)
        if bps <= 0:
            continue
        pbr_arr.append(round((asp[0][2] / bps), 2))
        name_arr.append(asp[0][3])
        date_arr.append(asp[0][1])
        code.append(stock_code)

    conn.close()
    pbr = pd.DataFrame(data = name_arr, index = code, columns=['name'])
    pbr['pbr'] = pbr_arr
    pbr['date'] = date_arr
    pbr = pbr.dropna()
    pbr = pbr[['pbr', 'date', 'name']]
    pbr = pbr.sort_values('pbr',ascending=True)
    
    return pbr

In [7]:
#getCombine 함수 (연간 데이터)
def getCombine(date):
    per = getPER(date) #해당 날짜의 per데이터 프레임을 구합니다.
    pbr = getPBR(date) #해당 날짜의 pbr데이터 프레임을 구합니다.
    
    combine = per #combine데이터 프레임에 per을 복사합니다.
    combine['combine'] = 0 #새로운 열을 만들고 per + pbr값을 저장합니다.
    combine['combine'] = per['per'] + pbr['pbr']
    combine['pbr'] = pbr['pbr'] #비교를 위해 pbr열을 넣어줍니다.
    combine = combine.dropna()
    combine = combine[['per', 'pbr', 'combine', 'date', 'name']]
    combine = combine.sort_values('combine',ascending=True) #combine값으로 정렬합니다.
    
    return combine

In [38]:
#getPERT 함수 (트레일링 데이터)
def getPERT():
    per_arr = []
    name_arr = []
    date_arr = []
    code = []
    for i in range(len(symbol_code)):
        conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
        curs = conn.cursor()
        stock_code = symbol_code[i]
        #트레일링 데이터를 사용할 때는 값이 한 개 밖에 없기 때문에 인자를 받지 않습니다.
        sql = f"SELECT stock_code, period, netinc1 FROM income_statement_trailing WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        income_statement = curs.fetchall()

        curs = conn.cursor()
        sql = f"SELECT stock_code, number FROM stock_number WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        stock_number = curs.fetchall()

        if len(income_statement) == 0:
            continue
        
        curs = conn.cursor()
        date = income_statement[0][1]
        sql = f"SELECT Code, Date, Close, Name FROM asp WHERE Code = '{stock_code}' AND Date = '{date}'"
        curs.execute(sql)
        asp = curs.fetchall()

        if len(asp) == 0:
            continue
        netinc1 = round(float(income_statement[0][2]) * 100000000, 2)
        eps = round(netinc1 / stock_number[0][1], 2)
        if eps <= 0:
            continue
        per_arr.append(round((asp[0][2] / eps), 2))
        name_arr.append(asp[0][3])
        date_arr.append(asp[0][1])
        code.append(stock_code)

    conn.close()
    per = pd.DataFrame(data = name_arr, index = code, columns=['name'])
    per['per'] = per_arr
    per['date'] = date_arr
    per = per.dropna()
    per = per[['per', 'date', 'name']]
    per = per.sort_values('per',ascending=True)
    
    return per

In [39]:
#getPBRT 함수 (트레일링 데이터)
def getPBRT():
    pbr_arr = []
    name_arr = []
    date_arr = []
    code = []
    for i in range(len(symbol_code)):
        conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
        curs = conn.cursor()
        stock_code = symbol_code[i]
        sql = f"SELECT stock_code, period, Controlling_Equity_Total FROM balance_statement_trailing WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        balance_statement = curs.fetchall()

        curs = conn.cursor()
        sql = f"SELECT stock_code, number FROM stock_number WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        stock_number = curs.fetchall()

        if len(balance_statement) == 0:
            continue
        
        curs = conn.cursor()
        date = balance_statement[0][1]
        sql = f"SELECT Code, Date, Close, Name FROM asp WHERE Code = '{stock_code}' AND Date = '{date}'"
        curs.execute(sql)
        asp = curs.fetchall()

        if len(asp) == 0:
            continue
        Controlling_Equity_Total = round(float(balance_statement[0][2]) * 100000000, 2)
        bps = round(Controlling_Equity_Total / stock_number[0][1], 2)
        if bps <= 0:
            continue
        pbr_arr.append(round((asp[0][2] / bps), 2))
        name_arr.append(asp[0][3])
        date_arr.append(asp[0][1])
        code.append(stock_code)

    conn.close()
    pbr = pd.DataFrame(data = name_arr, index = code, columns=['name'])
    pbr['pbr'] = pbr_arr
    pbr['date'] = date_arr
    pbr = pbr.dropna()
    pbr = pbr[['pbr', 'date', 'name']]
    pbr = pbr.sort_values('pbr',ascending=True)
    
    return pbr

In [40]:
#getCombineT 함수 (트레일링 데이터)
def getCombineT():
    per = getPERT()
    pbr = getPBRT()
    
    combine = per
    combine['combine'] = 0
    combine['combine'] = per['per'] + pbr['pbr']
    combine['pbr'] = pbr['pbr']
    combine = combine.dropna()
    combine = combine[['per', 'pbr', 'combine', 'date', 'name']]
    combine = combine.sort_values('combine',ascending=True)
    
    return combine

In [11]:
#연간 데이터로 저장된 데이터들의 per, pbr, combine으로 종목을 30개 추출합니다.
print("연간 데이터의 PER, PBR, Combine 종목 추출입니다.")
value = input("PER, PBR, Combine 중 선택해주세요. : ")
if value == 'PER':
    date = input("날짜를 입력해주세요 ex) 2021/12 : ")
    per = getPER(date)
    print(per.head(30))   #30개의 종목을 출력합니다.
elif value == 'PBR':
    date = input("날짜를 입력해주세요 ex) 2021/12 : ")
    pbr = getPBR(date)
    print(pbr.head(30))
elif value == 'Combine':
    date = input("날짜를 입력해주세요 ex) 2021/12 : ")
    combine = getCombine(date)
    print(combine.head(30))
else:
    print("잘못 입력하셨습니다.")

연간 데이터의 PER, PBR, Combine 종목 추출입니다.
PER, PBR, Combine 중 선택해주세요. : PER
날짜를 입력해주세요 ex) 2021/12 : 2020/12
         per     date      name
036830  0.66  2020/12   솔브레인홀딩스
016250  0.71  2020/12  SGC이테크건설
037350  0.79  2020/12     성도이엔지
009410  0.85  2020/12      태영건설
034810  0.95  2020/12      해성산업
032940  1.09  2020/12        원익
227420  1.63  2020/12     도부마스크
008060  1.66  2020/12        대덕
092230  1.82  2020/12    KPX홀딩스
004960  1.83  2020/12      한신공영
000700  1.87  2020/12     유수홀딩스
024800  1.89  2020/12    유성티엔에스
002020  2.00  2020/12       코오롱
040610  2.01  2020/12      SG&G
101930  2.05  2020/12      인화정공
014790  2.11  2020/12        한라
179720  2.16  2020/12      렌딩머신
004150  2.17  2020/12     한솔홀딩스
086460  2.29  2020/12  에스엔피제네틱스
900280  2.58  2020/12     골든센츄리
007120  2.59  2020/12    미래아이앤지
105840  2.62  2020/12        우진
013580  2.66  2020/12      계룡건설
009770  2.78  2020/12      삼정펄프
042420  2.81  2020/12   네오위즈홀딩스
035890  2.82  2020/12      서희건설
005820  2.83  2020/12        원림
0

In [14]:
#연간 데이터로 저장된 데이터들의 per, pbr, combine으로 종목을 30개 추출합니다.
print("연간 데이터의 PER, PBR, Combine 종목 추출입니다.")
value = input("PER, PBR, Combine 중 선택해주세요. : ")
if value == 'PER':
    date = input("날짜를 입력해주세요 ex) 2021/12 : ")
    per = getPER(date)
    print(per.head(30))
elif value == 'PBR':
    date = input("날짜를 입력해주세요 ex) 2021/12 : ")
    pbr = getPBR(date)
    print(pbr.head(30))
elif value == 'Combine':
    date = input("날짜를 입력해주세요 ex) 2021/12 : ")
    combine = getCombine(date)
    print(combine.head(30))
else:
    print("잘못 입력하셨습니다.")

연간 데이터의 PER, PBR, Combine 종목 추출입니다.
PER, PBR, Combine 중 선택해주세요. : PBR
날짜를 입력해주세요 ex) 2021/12 : 2020/12
         pbr     date       name
005010  0.14  2020/12        휴스틸
058650  0.16  2020/12      세아홀딩스
012320  0.17  2020/12     경동인베스트
003030  0.19  2020/12     세아제강지주
040610  0.19  2020/12       SG&G
900280  0.21  2020/12      골든센츄리
003830  0.22  2020/12       대한화섬
900110  0.23  2020/12  이스트아시아홀딩스
001430  0.23  2020/12    세아베스틸지주
000850  0.24  2020/12       화천기공
003300  0.24  2020/12      한일홀딩스
002240  0.25  2020/12       고려제강
024830  0.25  2020/12       세원물산
002030  0.25  2020/12        아세아
015760  0.25  2020/12       한국전력
005720  0.25  2020/12         넥센
017940  0.26  2020/12         E1
004690  0.26  2020/12        삼천리
071320  0.26  2020/12     지역난방공사
000860  0.26  2020/12     강남제비스코
067830  0.27  2020/12    세이브존I&C
002920  0.27  2020/12       유성기업
001940  0.27  2020/12   KISCO홀딩스
003200  0.27  2020/12       일신방직
003240  0.27  2020/12       태광산업
006660  0.27  2020/12       삼성공조
004970

In [15]:
#연간 데이터로 저장된 데이터들의 per, pbr, combine으로 종목을 30개 추출합니다.
print("연간 데이터의 PER, PBR, Combine 종목 추출입니다.")
value = input("PER, PBR, Combine 중 선택해주세요. : ")
if value == 'PER':
    date = input("날짜를 입력해주세요 ex) 2021/12 : ")
    per = getPER(date)
    print(per.head(30))
elif value == 'PBR':
    date = input("날짜를 입력해주세요 ex) 2021/12 : ")
    pbr = getPBR(date)
    print(pbr.head(30))
elif value == 'Combine':
    date = input("날짜를 입력해주세요 ex) 2021/12 : ")
    combine = getCombine(date)
    print(combine.head(30))
else:
    print("잘못 입력하셨습니다.")

연간 데이터의 PER, PBR, Combine 종목 추출입니다.
PER, PBR, Combine 중 선택해주세요. : Combine
날짜를 입력해주세요 ex) 2021/12 : 2020/12
         per   pbr  combine     date      name
037350  0.79  0.31     1.10  2020/12     성도이엔지
034810  0.95  0.54     1.49  2020/12      해성산업
032940  1.09  0.47     1.56  2020/12        원익
036830  0.66  0.91     1.57  2020/12   솔브레인홀딩스
016250  0.71  0.88     1.59  2020/12  SGC이테크건설
009410  0.85  0.74     1.59  2020/12      태영건설
008060  1.66  0.40     2.06  2020/12        대덕
092230  1.82  0.33     2.15  2020/12    KPX홀딩스
004960  1.83  0.32     2.15  2020/12      한신공영
040610  2.01  0.19     2.20  2020/12      SG&G
024800  1.89  0.37     2.26  2020/12    유성티엔에스
000700  1.87  0.49     2.36  2020/12     유수홀딩스
002020  2.00  0.41     2.41  2020/12       코오롱
101930  2.05  0.40     2.45  2020/12      인화정공
004150  2.17  0.31     2.48  2020/12     한솔홀딩스
014790  2.11  0.63     2.74  2020/12        한라
900280  2.58  0.21     2.79  2020/12     골든센츄리
086460  2.29  0.58     2.87  2020/12  에스엔피제네틱스


In [119]:
#트레일링 데이터로 저장된 데이터들의 per, pbr, combine으로 종목을 30개 추출합니다.
print("트레일링 데이터의 PER, PBR, Combine 종목 추출입니다.")
value = input("PER, PBR, Combine 중 선택해주세요. : ")
if value == 'PER':
    perT = getPERT()
    print(perT.head(30))
elif value == 'PBR':
    pbrT = getPBRT()
    print(pbrT.head(30))
elif value == 'Combine':
    combineT = getCombineT()
    print(combineT.head(30))
else:
    print("잘못 입력하셨습니다.")

트레일링 데이터의 PER, PBR, Combine 종목 추출입니다.
PER, PBR, Combine 중 선택해주세요. : PER
         per     date        name
386580  0.05  2022/03  한화플러스제2호스팩
001500  0.08  2022/03       현대차증권
019550  0.13  2022/03   SBI인베스트먼트
001510  0.29  2022/03        SK증권
007700  0.57  2022/03      F&F홀딩스
190650  1.04  2021/12   코리아에셋투자증권
095270  1.55  2022/03     웨이브일렉트로
104480  1.63  2022/03      티케이케미칼
005810  1.68  2022/03       풍산홀딩스
011200  1.71  2022/03         HMM
139480  1.72  2022/03         이마트
140520  1.79  2022/03        대창스틸
016250  1.79  2022/03    SGC이테크건설
024110  1.82  2022/03        기업은행
950130  1.87  2022/03      엑세스바이오
003460  1.99  2022/03        유화증권
001230  2.01  2022/03        동국제강
039560  2.04  2022/03      다산네트웍스
205470  2.06  2022/03        휴마시스
011300  2.08  2022/03          성안
000210  2.15  2022/03          DL
017940  2.15  2022/03          E1
004720  2.16  2022/03      팜젠사이언스
124560  2.21  2022/03       태웅로직스
001940  2.27  2022/03    KISCO홀딩스
005960  2.31  2022/03        동부건설
013580  2.

In [120]:
#트레일링 데이터로 저장된 데이터들의 per, pbr, combine으로 종목을 30개 추출합니다.
print("트레일링 데이터의 PER, PBR, Combine 종목 추출입니다.")
value = input("PER, PBR, Combine 중 선택해주세요. : ")
if value == 'PER':
    perT = getPERT()
    print(perT.head(30))
elif value == 'PBR':
    pbrT = getPBRT()
    print(pbrT.head(30))
elif value == 'Combine':
    combineT = getCombineT()
    print(combineT.head(30))
else:
    print("잘못 입력하셨습니다.")

트레일링 데이터의 PER, PBR, Combine 종목 추출입니다.
PER, PBR, Combine 중 선택해주세요. : PBR
         pbr     date       name
021820  0.04  2022/03       세원정공
900110  0.04  2022/03  이스트아시아홀딩스
012630  0.05  2022/03        HDC
012320  0.05  2022/03     경동인베스트
017940  0.06  2022/03         E1
058650  0.06  2022/03      세아홀딩스
005720  0.06  2022/03         넥센
005010  0.06  2022/03        휴스틸
024110  0.06  2022/03       기업은행
071320  0.06  2022/03     지역난방공사
015760  0.06  2022/03       한국전력
000850  0.06  2022/03       화천기공
900280  0.06  2022/03      골든센츄리
043370  0.07  2022/03     피에이치에이
023530  0.07  2022/03       롯데쇼핑
004840  0.07  2022/03      DRB동일
009200  0.07  2022/03      무림페이퍼
036530  0.07  2022/03     SNT홀딩스
002920  0.07  2022/03       유성기업
003300  0.07  2022/03      한일홀딩스
200880  0.07  2022/03       서연이화
004150  0.07  2022/03      한솔홀딩스
000860  0.07  2022/03     강남제비스코
037350  0.08  2022/03      성도이엔지
072710  0.08  2022/03      농심홀딩스
001940  0.08  2022/03   KISCO홀딩스
002240  0.08  2022/03       고려제강
0495

In [121]:
#트레일링 데이터로 저장된 데이터들의 per, pbr, combine으로 종목을 30개 추출합니다.
print("트레일링 데이터의 PER, PBR, Combine 종목 추출입니다.")
value = input("PER, PBR, Combine 중 선택해주세요. : ")
if value == 'PER':
    perT = getPERT()
    print(perT.head(30))
elif value == 'PBR':
    pbrT = getPBRT()
    print(pbrT.head(30))
elif value == 'Combine':
    combineT = getCombineT()
    print(combineT.head(30))
else:
    print("잘못 입력하셨습니다.")

트레일링 데이터의 PER, PBR, Combine 종목 추출입니다.
PER, PBR, Combine 중 선택해주세요. : Combine
         per   pbr  combine     date      name
001500  0.08  0.56     0.64  2022/03     현대차증권
001510  0.29  0.45     0.74  2022/03      SK증권
007700  0.57  0.18     0.75  2022/03    F&F홀딩스
005810  1.68  0.09     1.77  2022/03     풍산홀딩스
139480  1.72  0.10     1.82  2022/03       이마트
024110  1.82  0.06     1.88  2022/03      기업은행
104480  1.63  0.30     1.93  2022/03    티케이케미칼
095270  1.55  0.40     1.95  2022/03   웨이브일렉트로
140520  1.79  0.19     1.98  2022/03      대창스틸
016250  1.79  0.19     1.98  2022/03  SGC이테크건설
011200  1.71  0.39     2.10  2022/03       HMM
001230  2.01  0.15     2.16  2022/03      동국제강
017940  2.15  0.06     2.21  2022/03        E1
000210  2.15  0.09     2.24  2022/03        DL
039560  2.04  0.30     2.34  2022/03    다산네트웍스
001940  2.27  0.08     2.35  2022/03  KISCO홀딩스
011300  2.08  0.27     2.35  2022/03        성안
950130  1.87  0.52     2.39  2022/03    엑세스바이오
004720  2.16  0.28     2.44  20

In [41]:
#getPSRT 함수 (트레일링 데이터)
def getPSRT():
    psr_arr = []
    name_arr = []
    date_arr = []
    code = []
    for i in range(len(symbol_code)):
        conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
        curs = conn.cursor()
        stock_code = symbol_code[i]
        sql = f"SELECT stock_code, period, rev FROM income_statement_trailing WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        income_statement = curs.fetchall()

        curs = conn.cursor()
        sql = f"SELECT stock_code, number FROM stock_number WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        stock_number = curs.fetchall()

        if len(income_statement) == 0:
            continue
        
        curs = conn.cursor()
        date = income_statement[0][1]
        sql = f"SELECT Code, Date, Close, Name FROM asp WHERE Code = '{stock_code}' AND Date = '{date}'"
        curs.execute(sql)
        asp = curs.fetchall()

        if len(asp) == 0:
            continue
        rev = round(float(income_statement[0][2]) * 100000000, 2)
        sps = round(rev / stock_number[0][1], 2)
        if sps <= 0:
            continue
        psr_arr.append(round((asp[0][2] / sps), 2))
        name_arr.append(asp[0][3])
        date_arr.append(asp[0][1])
        code.append(stock_code)

    conn.close()
    psr = pd.DataFrame(data = name_arr, index = code, columns=['name'])
    psr['psr'] = psr_arr
    psr['date'] = date_arr
    psr = psr.dropna()
    psr = psr[['psr', 'date', 'name']]
    psr = psr.sort_values('psr',ascending=True)
    
    return psr

In [42]:
#getPCRT 함수 (트레일링 데이터)
def getPCRT():
    pcr_arr = []
    name_arr = []
    date_arr = []
    code = []
    for i in range(len(symbol_code)):
        conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
        curs = conn.cursor()
        stock_code = symbol_code[i]
        sql = f"SELECT stock_code, period, netinc1 FROM income_statement_trailing WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        income_statement = curs.fetchall()

        curs = conn.cursor()
        sql = f"SELECT stock_code, number FROM stock_number WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        stock_number = curs.fetchall()
        
        curs = conn.cursor()
        sql = f"SELECT stock_code, CFI_Total, CFI_In FROM cash_statement_trailing WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        cash_statement = curs.fetchall()

        if len(income_statement) == 0:
            continue
        if len(cash_statement) == 0:
            continue
        
        curs = conn.cursor()
        date = income_statement[0][1]
        sql = f"SELECT Code, Date, Close, Name FROM asp WHERE Code = '{stock_code}' AND Date = '{date}'"
        curs.execute(sql)
        asp = curs.fetchall()

        if len(asp) == 0:
            continue
        ocf = round(float(income_statement[0][2] + cash_statement[0][1] + cash_statement[0][2]) * 100000000, 2)
        cfps = round(ocf / stock_number[0][1], 2)
        if cfps <= 0:
            continue
        pcr_arr.append(round((asp[0][2] / cfps), 2))
        name_arr.append(asp[0][3])
        date_arr.append(asp[0][1])
        code.append(stock_code)

    conn.close()
    pcr = pd.DataFrame(data = name_arr, index = code, columns=['name'])
    pcr['pcr'] = pcr_arr
    pcr['date'] = date_arr
    pcr = pcr.dropna()
    pcr = pcr[['pcr', 'date', 'name']]
    pcr = pcr.sort_values('pcr',ascending=True)
    
    return pcr

In [43]:
#getAllCombineT 함수 (트레일링 데이터)
def getAllCombineT():
    per = getPERT()
    pbr = getPBRT()
    psr = getPSRT()
    pcr = getPCRT()
    
    combine = per
    combine['allcombine'] = 0
    combine['allcombine'] = per['per'] + pbr['pbr'] + psr['psr'] + pcr['pcr']
    combine['pbr'] = pbr['pbr']
    combine['psr'] = psr['psr']
    combine['pcr'] = pcr['pcr']
    combine = combine.dropna()
    combine = combine[['per', 'pbr', 'psr', 'pcr', 'allcombine', 'date', 'name']]
    combine = combine.sort_values('allcombine',ascending=True)
    
    return combine

In [44]:
#getPEVT 함수 (트레일링 데이터)
def getEVT():
    evEbitda_arr = []
    evSales_arr = []
    name_arr = []
    date_arr = []
    code = []
    for i in range(len(symbol_code)):
        conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
        curs = conn.cursor()
        stock_code = symbol_code[i]
        sql = f"SELECT stock_code, period, opr, rev FROM income_statement_trailing WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        income_statement = curs.fetchall()

        curs = conn.cursor()
        sql = f"SELECT stock_code, number FROM stock_number WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        stock_number = curs.fetchall()
        
        curs = conn.cursor()
        sql = f"SELECT stock_code, period, Current_Loans, Current_Portion_LT_Liab, LT_Bonds, LT_Loans, Cash_and_Cash_Equivalents FROM balance_statement_trailing WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        balance_statement = curs.fetchall()
        
        curs = conn.cursor()
        sql = f"SELECT stock_code, period, CFI_Total FROM cash_statement_trailing WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        cash_statement = curs.fetchall()

        if len(income_statement) == 0:
            continue
        if len(balance_statement) == 0:
            continue
        if len(cash_statement) == 0:
            continue
        
        curs = conn.cursor()
        date = income_statement[0][1]
        sql = f"SELECT Code, Date, Close, Name FROM asp WHERE Code = '{stock_code}' AND Date = '{date}'"
        curs.execute(sql)
        asp = curs.fetchall()

        if len(asp) == 0:
            continue
        mv = asp[0][2] * stock_number[0][1]
        ev = round(float(mv + (balance_statement[0][2] + balance_statement[0][3] + balance_statement[0][4] + balance_statement[0][5] - balance_statement[0][6]) * 100000000), 2)
        ebitda = round(float(income_statement[0][2] + cash_statement[0][2]) * 100000000, 2)
        sales = round(float(income_statement[0][3]) * 100000000, 2)
        if ev < 0:
            continue
        if ebitda <= 0:
            continue
        if sales <= 0:
            continue
        evEbitda_arr.append(round((ev / ebitda), 2))
        evSales_arr.append(round((ev / sales), 2))
        name_arr.append(asp[0][3])
        date_arr.append(asp[0][1])
        code.append(stock_code)

    conn.close()
    evEbitda = pd.DataFrame(data = name_arr, index = code, columns=['name'])
    evEbitda['evEbitda'] = evEbitda_arr
    evEbitda['date'] = date_arr
    evEbitda = evEbitda.dropna()
    evEbitda = evEbitda[['evEbitda', 'date', 'name']]
    evEbitda = evEbitda.sort_values('evEbitda',ascending=True)
    
    evSales = pd.DataFrame(data = name_arr, index = code, columns=['name'])
    evSales['evSales'] = evSales_arr
    evSales['date'] = date_arr
    evSales = evSales.dropna()
    evSales = evSales[['evSales', 'date', 'name']]
    evSales = evSales.sort_values('evSales',ascending=True)
    
    return evEbitda, evSales

In [13]:
#getPBRT 함수 (트레일링 데이터)
def getNcavT():
    ncav_arr = []
    name_arr = []
    date_arr = []
    code = []
    for i in range(len(symbol_code)):
        conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
        curs = conn.cursor()
        stock_code = symbol_code[i]
        sql = f"SELECT stock_code, period, Current_Assets_total, Liabilities_Total FROM balance_statement_trailing WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        balance_statement = curs.fetchall()

        if len(balance_statement) == 0:
            continue
        
        curs = conn.cursor()
        date = balance_statement[0][1]
        sql = f"SELECT Code, Date, Close, Name FROM asp WHERE Code = '{stock_code}' AND Date = '{date}'"
        curs.execute(sql)
        asp = curs.fetchall()

        if len(asp) == 0:
            continue
        Current_Assets_total = round(float(balance_statement[0][2]), 2)
        Liabilities_Total = round(float(balance_statement[0][3]), 2)
        ncav_arr.append(Current_Assets_total - Liabilities_Total)
        name_arr.append(asp[0][3])
        date_arr.append(asp[0][1])
        code.append(stock_code)

    conn.close()
    ncav = pd.DataFrame(data = name_arr, index = code, columns=['name'])
    ncav['ncav'] = ncav_arr
    ncav['date'] = date_arr
    ncav = ncav.dropna()
    ncav = ncav[['ncav', 'date', 'name']]
    ncav = ncav.sort_values('ncav',ascending=True)
    
    return ncav

In [1]:
#getMarginT 함수 (트레일링 데이터)
def getMarginT():
    gross_margin_arr = []
    operating_margin_arr = []
    profit_margin_arr = []
    name_arr = []
    date_arr = []
    code = []
    for i in range(len(symbol_code)):
        conn = pymysql.connect(host='localhost', user='root', password='root', db='mhdb', charset='utf8')
        curs = conn.cursor()
        stock_code = symbol_code[i]
        sql = f"SELECT stock_code, period, rev, gross, opr, netinc FROM income_statement_trailing WHERE stock_code = '{stock_code}'"
        curs.execute(sql)
        income_statement = curs.fetchall()

        if len(income_statement) == 0:
            continue
        if income_statement[0][2] == 0:
            continue
        
        curs = conn.cursor()
        date = income_statement[0][1]
        sql = f"SELECT Code, Date, Close, Name FROM asp WHERE Code = '{stock_code}' AND Date = '{date}'"
        curs.execute(sql)
        asp = curs.fetchall()

        if len(asp) == 0:
            continue
        gross_margin_arr.append(round((income_statement[0][3] / income_statement[0][2]), 2))
        operating_margin_arr.append(round((income_statement[0][4] / income_statement[0][2]), 2))
        profit_margin_arr.append(round((income_statement[0][5] / income_statement[0][2]), 2))
        name_arr.append(asp[0][3])
        date_arr.append(asp[0][1])
        code.append(stock_code)

    conn.close()
    gross_margin = pd.DataFrame(data = name_arr, index = code, columns=['name'])
    gross_margin['gross_margin'] = gross_margin_arr
    gross_margin['date'] = date_arr
    gross_margin = gross_margin.dropna()
    gross_margin = gross_margin[['gross_margin', 'date', 'name']]
    gross_margin = gross_margin.sort_values('gross_margin',ascending=True)
    
    operating_margin = pd.DataFrame(data = name_arr, index = code, columns=['name'])
    operating_margin['operating_margin'] = operating_margin_arr
    operating_margin['date'] = date_arr
    operating_margin = operating_margin.dropna()
    operating_margin = operating_margin[['operating_margin', 'date', 'name']]
    operating_margin = operating_margin.sort_values('operating_margin',ascending=True)
    
    profit_margin = pd.DataFrame(data = name_arr, index = code, columns=['name'])
    profit_margin['profit_margin'] = profit_margin_arr
    profit_margin['date'] = date_arr
    profit_margin = profit_margin.dropna()
    profit_margin = profit_margin[['profit_margin', 'date', 'name']]
    profit_margin = profit_margin.sort_values('profit_margin',ascending=True)
    
    return gross_margin, operating_margin, profit_margin

In [11]:
#트레일링 데이터로 저장된 데이터들의 per, pbr, combine으로 종목을 30개 추출합니다.
print("트레일링 데이터의 PER, PBR, Combine 종목 추출입니다.")
value = input("PER, PBR, Combine 중 선택해주세요. : ")
if value == 'PER':
    perT = getPERT()
    print(perT.head(30))
elif value == 'PBR':
    pbrT = getPBRT()
    print(pbrT.head(30))
elif value == 'PSR':
    psrT = getPSRT()
    print(psrT.head(30))
elif value == 'Combine':
    combineT = getCombineT()
    print(combineT.head(30))
else:
    print("잘못 입력하셨습니다.")

트레일링 데이터의 PER, PBR, Combine 종목 추출입니다.
PER, PBR, Combine 중 선택해주세요. : PSR
         psr     date      name
124500  0.03  2022/03      아이티센
122690  0.04  2022/03   서진오토모티브
011760  0.05  2022/03   현대코퍼레이션
001250  0.05  2022/03     GS글로벌
037400  0.06  2022/03  우리엔터프라이즈
038110  0.06  2022/03    에코플라스틱
000880  0.06  2022/03        한화
058220  0.06  2021/12       아리온
017940  0.06  2022/03        E1
050540  0.06  2022/03   한국코퍼레이션
001120  0.07  2022/03   LX인터내셔널
047050  0.07  2022/03  포스코인터내셔널
013520  0.07  2022/03   화승코퍼레이션
005990  0.07  2022/03     매일홀딩스
058650  0.07  2022/03     세아홀딩스
002020  0.07  2022/03       코오롱
032190  0.08  2022/03     다우데이타
001390  0.08  2022/03     KG케미칼
010100  0.08  2022/03     한국프랜지
024900  0.08  2022/03      덕양산업
037460  0.08  2022/03      삼지전자
101330  0.08  2022/03      모베이스
267290  0.08  2022/03    경동도시가스
001040  0.08  2022/03        CJ
007860  0.08  2022/03        서연
200880  0.08  2022/03      서연이화
010770  0.09  2022/03     평화홀딩스
012630  0.09  2022/03       HDC


In [13]:
#트레일링 데이터로 저장된 데이터들의 per, pbr, combine으로 종목을 30개 추출합니다.
print("트레일링 데이터의 PER, PBR, Combine 종목 추출입니다.")
value = input("PER, PBR, Combine 중 선택해주세요. : ")
if value == 'PER':
    perT = getPERT()
    print(perT.head(30))
elif value == 'PBR':
    pbrT = getPBRT()
    print(pbrT.head(30))
elif value == 'PSR':
    psrT = getPSRT()
    print(psrT.head(30))
elif value == 'PCR':
    pcrT = getPCRT()
    print(pcrT.head(30))
elif value == 'Combine':
    combineT = getCombineT()
    print(combineT.head(30))
else:
    print("잘못 입력하셨습니다.")

트레일링 데이터의 PER, PBR, Combine 종목 추출입니다.
PER, PBR, Combine 중 선택해주세요. : PCR
         pcr     date      name
007700  0.56  2022/03    F&F홀딩스
050540  0.86  2022/03   한국코퍼레이션
122690  0.88  2022/03   서진오토모티브
018500  0.89  2021/12      동원금속
001040  1.04  2022/03        CJ
139480  1.11  2022/03       이마트
068400  1.14  2022/03     SK렌터카
101330  1.18  2022/03      모베이스
005720  1.18  2022/03        넥센
095570  1.22  2022/03    AJ네트웍스
017940  1.24  2022/03        E1
036460  1.26  2022/03    한국가스공사
020560  1.27  2022/03    아시아나항공
001740  1.27  2022/03    SK네트웍스
034220  1.33  2022/03   LG디스플레이
200880  1.37  2022/03      서연이화
003380  1.43  2022/03      하림지주
089860  1.44  2022/03      롯데렌탈
007860  1.46  2022/03        서연
015750  1.47  2022/03     성우하이텍
005990  1.49  2022/03     매일홀딩스
019540  1.50  2022/03      일지테크
095270  1.52  2022/03   웨이브일렉트로
008060  1.54  2022/03        대덕
002030  1.56  2022/03       아세아
005810  1.57  2022/03     풍산홀딩스
037400  1.57  2022/03  우리엔터프라이즈
011200  1.58  2022/03       HMM


In [24]:
#트레일링 데이터로 저장된 데이터들의 per, pbr, combine으로 종목을 30개 추출합니다.
print("트레일링 데이터의 PER, PBR, Combine 종목 추출입니다.")
value = input("PER, PBR, Combine 중 선택해주세요. : ")
if value == 'PER':
    perT = getPERT()
    print(perT.head(30))
elif value == 'PBR':
    pbrT = getPBRT()
    print(pbrT.head(30))
elif value == 'PSR':
    psrT = getPSRT()
    print(psrT.head(30))
elif value == 'PCR':
    pcrT = getPCRT()
    print(pcrT.head(30))
elif value == 'Combine':
    combineT = getCombineT()
    print(combineT.head(30))
elif value == 'AllCombine':
    allcombineT = getAllCombineT()
    print(allcombineT.head(30))
else:
    print("잘못 입력하셨습니다.")

트레일링 데이터의 PER, PBR, Combine 종목 추출입니다.
PER, PBR, Combine 중 선택해주세요. : AllCombine
         per   pbr   psr   pcr  allcombine     date      name
007700  0.57  0.18  0.92  0.56        2.23  2022/03    F&F홀딩스
139480  1.72  0.10  0.15  1.11        3.08  2022/03       이마트
017940  2.15  0.06  0.06  1.24        3.51  2022/03        E1
016250  1.79  0.19  0.12  1.64        3.74  2022/03  SGC이테크건설
140520  1.79  0.19  0.24  1.72        3.94  2022/03      대창스틸
001230  2.01  0.15  0.20  1.60        3.96  2022/03      동국제강
005810  1.68  0.09  0.69  1.57        4.03  2022/03     풍산홀딩스
104480  1.63  0.30  0.88  1.60        4.41  2022/03    티케이케미칼
001940  2.27  0.08  0.16  1.92        4.43  2022/03  KISCO홀딩스
000210  2.15  0.09  0.50  1.71        4.45  2022/03        DL
002020  2.59  0.11  0.07  1.71        4.48  2022/03       코오롱
003030  2.36  0.11  0.16  1.85        4.48  2022/03    세아제강지주
011200  1.71  0.39  0.88  1.58        4.56  2022/03       HMM
036460  3.12  0.11  0.11  1.26        4.60  2022/03  

In [32]:
#트레일링 데이터로 저장된 데이터들의 per, pbr, combine으로 종목을 30개 추출합니다.
print("트레일링 데이터의 종목 추출입니다.")
value = input("PER, PBR, PSR, PCR, Combine, AllCombine, EV 중 선택해주세요. : ")
if value == 'PER':
    perT = getPERT()
    print(perT.head(30))
elif value == 'PBR':
    pbrT = getPBRT()
    print(pbrT.head(30))
elif value == 'PSR':
    psrT = getPSRT()
    print(psrT.head(30))
elif value == 'PCR':
    pcrT = getPCRT()
    print(pcrT.head(30))
elif value == 'Combine':
    combineT = getCombineT()
    print(combineT.head(30))
elif value == 'AllCombine':
    allcombineT = getAllCombineT()
    print(allcombineT.head(30))
elif value == 'EV':
    evEbitda, evSales = getEVT()
    print(evEbitda.head(30))
    print('------------------------------------------------------------')
    print(evSales.head(30))
else:
    print("잘못 입력하셨습니다.")

트레일링 데이터의 종목 추출입니다.
PER, PBR, PSR, PCR, Combine, AllCombine, EV 중 선택해주세요. : EV
        evEbitda     date      name
133750      0.03  2022/03      메가엠디
004720      0.05  2022/03    팜젠사이언스
054040      0.07  2022/03     한국컴퓨터
025560      0.23  2022/03      미래산업
007590      0.30  2022/03     동방아그로
241770      0.31  2022/03       메카로
003650      0.31  2022/03      미창석유
205470      0.34  2022/03      휴마시스
100700      0.50  2022/03     세운메디칼
002200      0.54  2022/03    한국수출포장
049430      0.60  2022/03       코메론
084010      0.63  2022/03      대한제강
015230      0.64  2022/03      대창단조
169330      0.70  2022/03      엠브레인
045300      0.72  2022/03     성우테크론
004150      0.80  2022/03     한솔홀딩스
253840      0.88  2022/03       수젠텍
061040      0.90  2022/03      알에프텍
071280      1.00  2022/03    로체시스템즈
011200      1.05  2022/03       HMM
352700      1.07  2022/03    씨앤투스성진
357230      1.20  2022/03     에이치피오
950190      1.22  2022/03       미투젠
049070      1.24  2022/03       인탑스
155650      1.27  202

In [37]:
#트레일링 데이터로 저장된 데이터들의 per, pbr, combine으로 종목을 30개 추출합니다.
print("트레일링 데이터의 종목 추출입니다.")
value = input("PER, PBR, PSR, PCR, Combine, AllCombine, EV, NCAV 중 선택해주세요. : ")
if value == 'PER':
    perT = getPERT()
    print(perT.head(30))
elif value == 'PBR':
    pbrT = getPBRT()
    print(pbrT.head(30))
elif value == 'PSR':
    psrT = getPSRT()
    print(psrT.head(30))
elif value == 'PCR':
    pcrT = getPCRT()
    print(pcrT.head(30))
elif value == 'Combine':
    combineT = getCombineT()
    print(combineT.head(30))
elif value == 'AllCombine':
    allcombineT = getAllCombineT()
    print(allcombineT.head(30))
elif value == 'EV':
    evEbitda, evSales = getEVT()
    print(evEbitda.head(30))
    print('------------------------------------------------------------')
    print(evSales.head(30))
elif value == 'NCAV':
    ncav = getNcavT()
    print(ncav.tail(30))
else:
    print("잘못 입력하셨습니다.")

트레일링 데이터의 종목 추출입니다.
PER, PBR, PSR, PCR, Combine, AllCombine, EV, NCAV 중 선택해주세요. : NCAV
        ncav(단위:억원)     date         name
096530     24895.90  2022/03           씨젠
026960     25575.28  2022/03           동서
003670     26716.50  2022/03       포스코케미칼
000100     27846.50  2022/03         유한양행
036530     28663.00  2022/03       SNT홀딩스
029530     31143.57  2022/03         신도리코
090430     31319.50  2022/03       아모레퍼시픽
001940     37777.53  2022/03     KISCO홀딩스
272210     38272.80  2022/03        한화시스템
036570     45381.70  2022/03        엔씨소프트
035250     45940.70  2022/03         강원랜드
003240     46892.40  2022/03         태광산업
068270     51912.50  2022/03         셀트리온
285130     52143.20  2022/03        SK케미칼
294870     52336.00  2022/03    HDC현대산업개발
009150     59895.00  2022/03         삼성전기
002790     64413.20  2022/03         아모레G
137310     67533.50  2022/03     에스디바이오센서
091990     71941.00  2022/03     셀트리온헬스케어
003550     72767.00  2022/03           LG
011170     76453.00  2022/03   

In [15]:
#트레일링 데이터로 저장된 데이터들의 per, pbr, combine으로 종목을 30개 추출합니다.
print("트레일링 데이터의 종목 추출입니다.")
value = input("PER, PBR, PSR, PCR, Combine, AllCombine, EV, NCAV, Margin 중 선택해주세요. : ")
if value == 'PER':
    perT = getPERT()
    print(perT.head(30))
elif value == 'PBR':
    pbrT = getPBRT()
    print(pbrT.head(30))
elif value == 'PSR':
    psrT = getPSRT()
    print(psrT.head(30))
elif value == 'PCR':
    pcrT = getPCRT()
    print(pcrT.head(30))
elif value == 'Combine':
    combineT = getCombineT()
    print(combineT.head(30))
elif value == 'AllCombine':
    allcombineT = getAllCombineT()
    print(allcombineT.head(30))
elif value == 'EV':
    evEbitda, evSales = getEVT()
    print(evEbitda.head(30))
    print('------------------------------------------------------------')
    print(evSales.head(30))
elif value == 'NCAV':
    ncav = getNcavT()
    print(ncav.tail(30))
elif value == 'Margin':
    gross_margin, operating_margin, profit_margin = getMarginT()
    print(gross_margin.tail(30))
    print('------------------------------------------------------------')
    print(operating_margin.tail(30))
    print('------------------------------------------------------------')
    print(profit_margin.tail(30))
else:
    print("잘못 입력하셨습니다.")

트레일링 데이터의 종목 추출입니다.
PER, PBR, PSR, PCR, Combine, AllCombine, EV, NCAV, Margin 중 선택해주세요. : Margin
        gross_margin     date      name
030200          1.00  2022/03        KT
289220          1.00  2022/03    자이언트스텝
058850          1.00  2022/03      KTcs
089600          1.00  2022/03     나스미디어
300080          1.00  2022/03       플리토
377300          1.00  2022/03     카카오페이
293490          1.00  2022/03    카카오게임즈
069080          1.00  2022/03        웹젠
035720          1.00  2022/03       카카오
032640          1.00  2022/03    LG유플러스
112040          1.00  2022/03      위메이드
101730          1.00  2022/03    위메이드맥스
123420          1.00  2022/03   위메이드플레이
075130          1.00  2022/03      플랜티넷
299900          1.00  2022/03   위지윅스튜디오
234340          1.00  2022/03      세틀뱅크
284620          1.00  2022/03    카이노스메드
094360          1.00  2022/03    칩스앤미디어
263050          1.00  2022/03      유틸렉스
377450          1.00  2022/03       리파인
377480          1.00  2022/03      마인즈랩
067920          1.00  2